In [60]:
import pandas as pd
course_data = pd.read_csv('training_pd.csv')
user_input = ''

In [61]:
course_data['label'] = course_data['label'].astype('int')

In [62]:
course_data['full_text'] = course_data['Job']+course_data['Course_Name']+course_data['Course_Hard_Skills']

In [63]:
course_data.head()

,Job,Softskills,Hardskills,Course_Name,Course_Description,Course_Hard_Skills,Course_Soft_Skills,label,full_text
0,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Case studies in corporate finance,structured around the most important financial...,"financial analysis, decision making, risk asse...",NaN,1,Mortgage AdvisorCase studies in corporate fina...
1,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Behavioural and sociological finance,covers current developments in finance. Possib...,"Understanding current developments in finance,...",NaN,1,Mortgage AdvisorBehavioural and sociological f...
2,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Senior seminar in economics and finance,applies core theoretical knowledge in economic...,"research skills, theoretical knowledge in econ...",NaN,1,Mortgage AdvisorSenior seminar in economics an...
3,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Current issues in asset management and private...,seeks to cover current development and current...,"Asset management, Private banking, Identifying...",NaN,1,Mortgage AdvisorCurrent issues in asset manage...
4,Mortgage Advisor,"Communication skills, active listening, proble...",Knowledge of mortgage products and regulations...,Valuation using financial statements,o introduce students to the concepts of financ...,Basic skills and techniques to analyze financi...,NaN,1,Mortgage AdvisorValuation using financial stat...


In [83]:
from transformers import DistilBertTokenizerFast
from collections import defaultdict
dataset = defaultdict(dict)
train = dict()
val = dict()
train['text'] = list(course_data['full_text'][:100])
val['text'] = list(course_data['full_text'][100:])
train['label'] = list(course_data['label'][:100])
val['label'] = list(course_data['label'][100:])
dataset['train'] = train
dataset['val'] = val

In [84]:
import torch
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(dataset['train']['text'], truncation=True, padding=True)
val_encodings = tokenizer(dataset['val']['text'], truncation=True, padding=True)
class tokenized_dataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __getitem__(self, idx):
        item = {'input_ids': torch.tensor(self.data['input_ids'][idx]),
                'attention_mask': torch.tensor(self.data['attention_mask'][idx])}
        item['label'] = torch.tensor(self.label[idx])
        return item
    def __len__(self):
        return len(self.label)
    
small_tokenized_dataset = defaultdict()
small_tokenized_dataset['train']=tokenized_dataset(train_encodings,list(dataset['train']['label']))
small_tokenized_dataset['val'] =tokenized_dataset(val_encodings,list(dataset['val']['label']))

small_tokenized_dataset['train'][:2]

{'input_ids': tensor([[  101, 14344,  8619, 18382,  2913,  1999,  5971,  5446, 16294,  2319,
          13247,  4106,  1010,  3247,  2437,  1010,  3891,  7667,  1010,  3007,
           3252,  4106,  1010,  5211, 26004,  1010, 10067,  3006,  4106,  1010,
           5971, 10615,  4106,  1010,  5356,  4834,  2968,  1998,  4106,  1010,
           1049,  1004,  1037,  4106,  1012,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0],
         [  101, 14344,  8619,  4783,  3270, 25500, 11137,  1998, 24846,  5446,
          20824, 24911,  2783,  8973,  1999,  5446,  1010,  3716,  1997,  3036,
           6202,  1998,  3006,  2437,  1010,  4

In [104]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import os 
from transformers import DistilBertForSequenceClassification
import torch
from torch.utils.data import DataLoader

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
# 设置随机数种子
# ------------------------------------------------------------------------------------------------------------------------------
# Write your code here
import datetime
train_data_loader = DataLoader(small_tokenized_dataset['train'],batch_size=12,shuffle=True)
val_data_loader = DataLoader(small_tokenized_dataset['val'],batch_size=12)
# Define your model. optimizer, hyper-parameter and etc.
from sklearn.metrics import accuracy_score
distil_bert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1,problem_type="binary_label_classification")
optimizer = AdamW(distil_bert_model.parameters(),betas=(0.9,0.999),lr=5e-5,eps=1e-5,weight_decay=1e-3)
num_epochs = 8
distil_bert_model.cuda()
bceloss = torch.nn.BCELoss()
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=3, num_training_steps=num_epochs)
start_time = datetime.datetime.now()
for epoch in tqdm(range(num_epochs)):
    #train and evaluate your model
    # training
    distil_bert_model.train()
    train_y_pred = []
    train_y_true = []
    for step, batch in enumerate(train_data_loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['label'].cuda()
        y_pred = distil_bert_model(input_ids,attention_mask=attention_mask, labels=labels)
        y_pred = torch.sigmoid(y_pred.logits).squeeze().float()
        loss = bceloss(y_pred, labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_y_pred.extend((y_pred>0.5).tolist())
        train_y_true.extend(batch['label'].tolist())
    train_acc = accuracy_score(train_y_pred, train_y_true)
    # evaluation
    distil_bert_model.eval()
    val_y_pred = []
    val_y_true = []
    for step, batch in enumerate(val_data_loader):
        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['label'].cuda()
        y_pred = distil_bert_model(input_ids,attention_mask=attention_mask, labels=labels)
        y_pred = torch.sigmoid(y_pred.logits).squeeze().float()
        val_y_pred.extend((y_pred>0.5).tolist())
        val_y_true.extend(batch['label'].tolist())
    validation_acc = accuracy_score(val_y_pred, val_y_true)
    scheduler.step()
    # print the training process
    print("Epoch {}: train acc = {:.4f}, validation acc = {:.4f}".format(epoch + 1, train_acc, validation_acc))
end_time = datetime.datetime.now()
print('distilbert training time used:',(end_time-start_time).seconds)
torch.save(distil_bert_model.state_dict(),'pytorch_model.pth')
# ------------------------------------------------------------------------------------------------------------------------------

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1: train acc = 0.5900, validation acc = 0.6809
Epoch 2: train acc = 0.6000, validation acc = 0.6809
Epoch 3: train acc = 0.6400, validation acc = 0.8319
Epoch 4: train acc = 0.9000, validation acc = 0.8702
Epoch 5: train acc = 0.9600, validation acc = 0.8936
Epoch 6: train acc = 0.9800, validation acc = 0.8787
Epoch 7: train acc = 1.0000, validation acc = 0.9000
Epoch 8: train acc = 1.0000, validation acc = 0.9170
distilbert training time used: 7


In [105]:
# inference

In [126]:
courses = pd.read_csv('output_cleaned.csv')
course_skill_dict = courses[['Course_Name','Course_Hard_Skills']].set_index('Course_Name').to_dict()

In [144]:
inference = []
user_input = 'economist'
courses.dropna(subset=['Course_Hard_Skills'],inplace=True,axis=0)
courses = courses.reset_index(drop=True)
for i in range(courses.shape[0]):
    
    input_joined = user_input+' '.join(list(courses.loc[i,['Course_Name','Course_Hard_Skills']]))
    #print(input_joined)
    inference.append(input_joined)
#4
import numpy as np
inference = tokenizer(inference,padding=True,truncation=True)
inference = tokenized_dataset(inference,label=np.ones(len(inference['input_ids'])))
infer_loader = DataLoader(inference,batch_size=24,shuffle=False)

In [145]:
inf_y_pred = []
distil_bert_model.eval()
with torch.no_grad():
    for step, batch in enumerate(infer_loader):
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            y_pred = distil_bert_model(input_ids,attention_mask=attention_mask)
            inf_y_pred.extend(torch.sigmoid(y_pred.logits[:,0]).tolist())
_,index = torch.topk(torch.tensor(inf_y_pred),10)
course_name = courses.loc[index,'Course_Name'].values
skills = []
for c in course_name:
    skills.append(course_skill_dict['Course_Hard_Skills'][c])
print('skill learned:',skills[0])
print()
print('courses recommended:')
for i,c in enumerate(course_name):
    print(i,',',c)

skill learned: Investment consulting industry, Asset management, Private wealth industry, Institutional investors, Family office, Practical understanding, Industry relevance.

courses recommended:
0 , Investment consulting, institutional business and family office
1 , Project management
2 , Current issues in asset management and private banking industr
3 , Behavioural and sociological finance
4 , Risk management
5 , Valuation using financial statements
6 , The political economy of globalization
7 , Private banking and wealth management
8 , Senior seminar in economics and finance
9 , Management control
